# Alpaca

Registrarse:


https://alpaca.markets/

Documentacion:


https://alpaca.markets/docs/

In [1]:
import requests

In [2]:
from keys_alpaca import *

In [3]:
def getAccount():    
    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/account"
    
    r = requests.get(url = endpoint, headers = headers)
    
    js = r.json()
    return js

Veamos entonces lo que nos devuelve:

In [4]:
cuenta  = getAccount()

cuenta.get('cash')

'100000'

In [5]:
cuenta.keys()

dict_keys(['id', 'account_number', 'status', 'currency', 'buying_power', 'regt_buying_power', 'daytrading_buying_power', 'cash', 'portfolio_value', 'pattern_day_trader', 'trading_blocked', 'transfers_blocked', 'account_blocked', 'created_at', 'trade_suspended_by_user', 'multiplier', 'shorting_enabled', 'equity', 'last_equity', 'long_market_value', 'short_market_value', 'initial_margin', 'maintenance_margin', 'last_maintenance_margin', 'sma', 'daytrade_count'])

## MarketData

Vamos a ver como pedimos marketData con esta API, empecemos por la data histórica, le vamos a tener que pasar dos parámetros a la API:
   * TimeFrame
       * 1Min
       * 5Min
       * 15Min
       * 1D (day)
   * Symbolo/Simbolos

In [6]:
tickers = 'AAPL,MSFT'
tickers = tickers.split(",")
tickers

['AAPL', 'MSFT']

In [7]:
lista = ['GGAL','AAPL','AMZN']
','.join(lista)

'GGAL,AAPL,AMZN'

In [8]:
import datetime as dt
import pandas as pd

def getHistory(symbol, start, end, timeframe='1D'):    
    
    start = dt.datetime.strftime(start, format='%Y-%m-%dT%T-04:00')
    end = dt.datetime.strftime(end, format='%Y-%m-%dT%T-04:00')

    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params =  {'symbols' : symbol, 'start':start, 'end':end}
    endpoint = "https://data.alpaca.markets/v1/bars/"+timeframe
    
    r = requests.get(url = endpoint, headers =headers, params=params)
    
    js = r.json()
    
    tickers = symbol.split(",")
    dfs = {}
    for ticker in tickers:
        dfs[ticker] = pd.DataFrame(js[ticker])
        dfs[ticker].t = pd.to_datetime(dfs[ticker].t, unit='s')
    return dfs

In [9]:
start = dt.datetime(2020,7,1,0,0,0)
end = dt.datetime(2020,10,1,0,0,0)
dfs = getHistory('AAPL,MSFT', start, end, timeframe='day')
dfs['AAPL']

,t,o,h,l,c,v
0,2020-07-01 04:00:00,365.12,367.36,363.91,364.01,25429807
1,2020-07-02 04:00:00,367.85,370.47,363.65,364.12,25332853
2,2020-07-06 04:00:00,370.00,375.78,369.87,373.81,26084791
3,2020-07-07 04:00:00,375.41,378.62,372.23,372.64,24610596
4,2020-07-08 04:00:00,376.72,381.50,376.36,381.44,26025692
...,...,...,...,...,...,...
60,2020-09-25 04:00:00,108.43,112.44,107.67,112.28,137542199
61,2020-09-28 04:00:00,114.98,115.32,112.78,115.00,119572328
62,2020-09-29 04:00:00,114.55,115.31,113.57,114.11,89125872
63,2020-09-30 04:00:00,113.77,117.25,113.62,115.61,115978209


In [10]:
dfs['AAPL']

,t,o,h,l,c,v
0,2020-07-01 04:00:00,365.12,367.36,363.91,364.01,25429807
1,2020-07-02 04:00:00,367.85,370.47,363.65,364.12,25332853
2,2020-07-06 04:00:00,370.00,375.78,369.87,373.81,26084791
3,2020-07-07 04:00:00,375.41,378.62,372.23,372.64,24610596
4,2020-07-08 04:00:00,376.72,381.50,376.36,381.44,26025692
...,...,...,...,...,...,...
60,2020-09-25 04:00:00,108.43,112.44,107.67,112.28,137542199
61,2020-09-28 04:00:00,114.98,115.32,112.78,115.00,119572328
62,2020-09-29 04:00:00,114.55,115.31,113.57,114.11,89125872
63,2020-09-30 04:00:00,113.77,117.25,113.62,115.61,115978209


## Ultimo Trade de un ticker

In [11]:
def lastTade(symbol):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://data.alpaca.markets/v1/last/stocks/"+symbol
    r = requests.get(url = endpoint, headers = headers)
    js = r.json()['last']
    
    js['timestamp'] = pd.to_datetime(js['timestamp'])
    return js
    
lastTade("AAPL")

{'price': 121.5,
 'size': 38,
 'exchange': 15,
 'cond1': 37,
 'cond2': 0,
 'cond3': 0,
 'cond4': 0,
 'timestamp': Timestamp('2020-10-14 19:19:10.099000')}

## Ultimo dato del libro

In [12]:
import pandas as pd

def lastQuote(symbol):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://data.alpaca.markets/v1/last_quote/stocks/"+symbol
    r = requests.get(url = endpoint, headers = headers)
    js = r.json()['last']
    js['timestamp'] = pd.to_datetime(js['timestamp'])
    return js
    
lastQuote("AAPL")

{'askprice': 121.48,
 'asksize': 4,
 'askexchange': 15,
 'bidprice': 121.46,
 'bidsize': 4,
 'bidexchange': 15,
 'timestamp': Timestamp('2020-10-14 19:19:15.301000')}

# Ruteo de ordenes

## Orden de Compra/Venta  (POST)

In [13]:
def order(symbol, qty, side, tipo, time_in_force, limit_price ):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'symbol':symbol, 
              'qty':qty, 
              'side':side, 
              'type':tipo,
              'time_in_force':time_in_force,
              'limit_price':limit_price, 
              'order_class':'simple'}
    
    r = requests.post(url = "https://paper-api.alpaca.markets/v2/orders", 
                      headers = headers, json = params)
    
    js = r.json()
    return js

In [14]:
lastQuote("GGAL")

{'askprice': 7.49,
 'asksize': 7,
 'askexchange': 17,
 'bidprice': 7.34,
 'bidsize': 7,
 'bidexchange': 17,
 'timestamp': Timestamp('2020-09-28 14:08:05.574000')}

In [15]:
lastTade("GGAL")

{'price': 6.57,
 'size': 100,
 'exchange': 17,
 'cond1': 0,
 'cond2': 0,
 'cond3': 0,
 'cond4': 0,
 'timestamp': Timestamp('2020-10-14 19:19:10.296000')}

## Envio varias órdenes

In [16]:
tickers = ['PAM', 'BBAR', 'TGS', 'CRESY', 'MELI']
monto = 10000

for ticker in tickers:
    px  = lastQuote(ticker)['bidprice'] * 1.001
    q = monto // px

    # orden = order(symbol=ticker, qty=q, side='buy', tipo='limit', time_in_force='gtc',limit_price = px )
    print(px, ticker)

11.411399999999999 PAM
2.42242 BBAR
4.594589999999999 TGS
3.0130099999999995 CRESY
1238.7975599999997 MELI


## Consulta de estado general

In [17]:
def orders(status="all", limit=500): 
    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'status':status, 
              'limit':limit}
    
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/orders", 
                      headers = headers, params = params)

    js = r.json()
    df = pd.DataFrame(js)
    return df

listaOrdenes = orders(status='open')
listaOrdenes

""


## Consulta puntual  (GET)

In [18]:
def getOrder(orderID):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    base = "https://paper-api.alpaca.markets/v2/orders/"
    r = requests.get(url = base+orderID, headers = headers)
    
    js = r.json()
    return js

## Modificación de orden  (PATCH)

In [19]:
def replaceOrder(order_id, qty=None, time_in_force=None, limit_price=None):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'qty':qty, 
              'time_in_force':time_in_force,
              'limit_price':limit_price}
    
    endpoint = "https://paper-api.alpaca.markets/v2/orders/"+order_id
    r = requests.patch(url = endpoint, headers = headers, json = params)
    
    js = r.json()
    return js

## Eliminar orden (DELETE)

In [20]:
def cancelOrder(order_id):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/orders/"+order_id
    r = requests.delete(url = endpoint,headers = headers)

## Ver Posiciones Abiertas de la cuenta

In [21]:
def positions():    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/positions"
    r = requests.get(url = endpoint, headers = headers)
    js = r.json()
    
    if len(js):
        df = pd.DataFrame(positions())
        df = df.loc[:,['symbol','qty','side','avg_entry_price','market_value',
                       'cost_basis','unrealized_pl','current_price',
                       'lastday_price']]
    else:
        df = pd.DataFrame()
    return df
    
portf = positions()
portf

""


# Screener

## Listado de acitivos

In [22]:
def assets(only_shorteables=False):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/assets", 
                      headers = headers)

    js = r.json()
    df = pd.DataFrame(js)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    
    if only_shorteables:
        df= df.loc[df.shortable==True]
        
    return df

In [23]:
activos = assets(only_shorteables=True)
activos

,id,exchange,symbol,name,shortable
0,b65e1cd5-0682-4eb3-844c-8901712b9cc4,NYSE,BWA,BorgWarner Inc.,True
1,d1d3b51c-772b-4eb8-9008-efb73b179e57,AMEX,BTN,"Ballantyne Strong, Inc",True
2,3463981c-fe71-4351-b677-ab12f87c3f2e,AMEX,DLA,Delta Apparel Inc.,True
3,db432267-e34d-47d4-be28-83ac76df4751,NYSE,EPR,EPR Properties,True
5,d4c735dd-ab6a-4f96-ad59-fc2e2dbff6c9,NYSE,IBN,ICICI Bank Limited,True
...,...,...,...,...,...
10244,675f7911-9aca-418a-acd5-07cfacb9d32b,ARCA,GLD,"SPDR Gold Trust, SPDR Gold Shares",True
10249,8ccae427-5dd0-45b3-b5fe-7ba5e422c766,NASDAQ,TSLA,"Tesla, Inc. Common Stock",True
10251,fd03066c-16d8-4eff-8c72-16ce1c44e38d,NYSE,VALE,VALE S.A.,True
10253,b0b6dd9d-8b9b-48a9-ba46-b9d54906e415,NASDAQ,AAPL,Apple Inc. Common Stock,True


In [24]:
activos.groupby('exchange').size()

exchange
AMEX        76
ARCA       443
BATS        49
NASDAQ    1995
NYSE      1833
dtype: int64

In [25]:
activos.groupby('shortable').size()

shortable
True    4396
dtype: int64

## WatchLists

## CRUD

### Crear listas

In [26]:
def createWatchlist(name, symbols):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'name':name, 'symbols':symbols}
    r = requests.post(url = "https://paper-api.alpaca.markets/v2/watchlists", 
                      headers = headers, json=params)
    
    js = r.json()
    df = pd.DataFrame(js['assets'])
    df.set_index('symbol', inplace=True)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    return js['id'], df

### Ver listas creadas

In [27]:
def watchlist():    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/watchlists", 
                      headers = headers)

    js = r.json()
    df = pd.DataFrame(js)
    try:
        df.drop('account_id',axis=1, inplace=True)
    except:
        pass
    
    return df

### Obtener watchList xID

In [28]:
def getWatchlist(wID):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    base = 'https://paper-api.alpaca.markets/v2/watchlists/'
    r = requests.get(url = base + wID, headers = headers)

    js = r.json()
    df = pd.DataFrame(js['assets'])
    df.set_index('symbol', inplace=True)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    
    return df

### Borrar watchList xID

In [29]:
def deleteWatchlist(wID):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/watchlists/"+wID
    r = requests.delete(url = endpoint, headers = headers)
    js = r.text
    return js

### Modificar watchList xID (PUT)

In [30]:
def updateWatchlist(wID, lista):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'symbols': lista}
    endpoint = "https://paper-api.alpaca.markets/v2/watchlists/"+wID
    r = requests.put(url = endpoint, headers = headers, json=params)
    
    return r

### Agregar Ticker a una watchList existente (POST)

In [31]:
def addTicker(wID, symbol):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'symbol': symbol}
    endpoint = "https://paper-api.alpaca.markets/v2/watchlists/"+wID
    r = requests.post(url = endpoint, headers = headers, json=params)
    js = r.json()
    df = pd.DataFrame(js['assets'])
    df.set_index('symbol', inplace=True)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    
    return df

# Ejercicios - Screener

Atención, la idea es terminar los 7 ejercicios porque están pensado como un bloque, es decir que todos forman parte de una especie de mini-screener para poder hacer una intro a este tema, aprovechando la funcionalidad de las watchlists de Alpaca podemos armar nuestro primer screener de 9000 activos de USA en 2 minutos via señal de trading predefinida y guardar los resultados como si se tratara de una alerta de trading

1- Partiendo de la función vista en clase para crear una watchList, mejorar la función para que (sin dejar de devolver el id y la lista en caso de estar todo ok), en caso de que se pase un ticker no listado devuelva al usuario el mensaje de error. Verificar su funcionamiento

2- Vamos a empezar a pensar en un potencial screener, para ello vamos a usar la función "assets" para armar otra función que me devuelva todos los tickers de la API y luego de ello con esos activos vamos a armar una gran lista con listas de a 200 activos, ya que la API me  permite ver series históricas trayendo de a 200 activos, para verificar el script vamos a imprimir la última lista y su cantidad de activos que debe ser menor de 200 y la cantidad de listas de 200
<br><br>A fines de poder usar este código mas adelante en un screener meter todo el código en una función: listasActivos()

3- Siguiendo en la línea de ir preparando el terreno para llegar a un screener, armar una función que si le paso una lista con listas de tickers me devuelva una lista con strings de tickers separados por comas, es decir:
 * Recibe: [ ['activo1','activo2','activo3'], ['activo4','activo5','activo_nn'] ]
 * Devuelve: [ 'activo1,activo2,activo3', 'activo4,activo5,activo_nn' ]
 
 Probar la función dando como entrada la salida de la función del ejercicio anterior

4- Agregarle lo necesario a prueba de fallos de la funcion getHistory definida en este capítulo para que acepte una lista de activos y me devuelve un diccionario cuyas claves sean las listas del ejercicio anterior y los valores unos dataFrames con el historial

Verificar el funcionamiento correcto de esta función pasándole como argumento la primera lista de 200 activos del ejercicio anterior en un timeframe de 1 minuto de los ultimos 60 dias

5- Seguimos armando el screener de a poco, paso a paso como quien no quiere la cosa los voy a ir llevando, confien en mi...

crear una función que tome de entrada un dataFrame de datos y me devuelva como salida un string que indique en funcion del MACD si da señal alcista o bajista:
   * Alcista: Si el histograma del MACD pasa de negativo a positivo
   * Bajista: Si el histograma del MACD pasa de positivo a negativo
   

6- Usando las funciones necesarias de los ejercicio anteriores crear un script que tome todos los activos de la API y me cree una watchList llamada "Shorts" y otra llamada "Longs" con los activos que obviamente cumplan las condiciones enunciadas para tal configuración en el ejercicio previo

7- Armar un script que opere una watchlist

En este caso:

    la watchlist "Longs" las va a comprar a precio de mercado
    la watchlilst "Shorts" las va a vender a precio de mercado

# Respuestas

In [32]:
import pandas as pd

In [33]:
#========================#
#      Ejercicio 1       #
#========================#


def createWatchlist(name, symbols):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'name':name, 'symbols':symbols}
    r = requests.post(url = "https://paper-api.alpaca.markets/v2/watchlists", 
                      headers = headers, json=params)
    
    js = r.json()
    try:
        msg = js['id']
        df = pd.DataFrame(js['assets'])
        df.set_index('symbol', inplace=True)
        df = df.loc[df.status=='active']
        df =df.drop(['status','class','easy_to_borrow','tradable','marginable'],axis=1)
    except:
        df = pd.DataFrame()
        msg = js
    return msg, df

tickers = ['BBAR','BMA', 'Y_VA_EL_TERCERO']
createWatchlist('lista_test', symbols = tickers)

({'code': 40010001, 'message': 'asset not found: Y_VA_EL_TERCERO'},
 Empty DataFrame
 Columns: []
 Index: [])

In [47]:
#========================#
#      Ejercicio 2       #
#========================#

def listasActivos():    
    activos = assets(only_shorteables=True)
    tickers = activos.symbol.to_list()

    listas = []
    for i in range(len(tickers)//200 + 1):
        listas.append(tickers[i*200:(i+1)*200])

    return listas

listas = listasActivos()

print('\n Ultima lista:\n',listas[len(listas)-1])
print('\n Cantidad de Ultima lista:',len(listas[len(listas)-1]))
print('\n Cantidad de listas de 200:',len(listas)-1)


 Ultima lista:
 ['ZTR', 'ZNGA', 'ZTO', 'ZTS', 'ZUO', 'ZYME', 'ZYXI', 'NVGS', 'ESTE', 'METC', 'CLDT', 'TALO', 'APH', 'ASIX', 'DE', 'DVN', 'EQC', 'AAN', 'ICLR', 'KBE', 'M', 'OKE', 'RACE', 'SM', 'VRS', 'ZUMZ', 'WPX', 'NUGT', 'CRK', 'NESR', 'PBR', 'WLL', 'CPE', 'SBOW', 'AINC', 'ARMK', 'AXS', 'BOX', 'CUZ', 'DOC', 'ECL', 'H', 'HUBS', 'KWR', 'AACQU', 'NPO', 'VERX', 'VVI', 'AAT', 'ASPU', 'ACIA', 'AHT', 'ALLO', 'ALNY', 'ARKK', 'ASND', 'ASPN', 'AVID', 'AVNW', 'BAP', 'ABBV', 'BHVN', 'BIO', 'BLUE', 'BMA', 'BR', 'AUY', 'BRO', 'BRX', 'BSAC', 'BSBR', 'BSCM', 'AACQ', 'AIZ', 'ALLY', 'ALLT', 'AMTI', 'ASML', 'ASRV', 'AVLR', 'AXSM', 'BGS', 'ABT', 'BRG', 'BRBR', 'BRC', 'BRKS', 'BRMK', 'BRP', 'BRZU', 'BSCK', 'BSCO', 'BRFS', 'CIT', 'CLS', 'CORE', 'CVE', 'DDOG', 'DGX', 'EIGR', 'EQT', 'EYEN', 'GLUU', 'GOGL', 'GTS', 'JELD', 'KMB', 'LGF.A', 'MAGS', 'MAR', 'MHLD', 'MOH', 'NET', 'NSA', 'NWPX', 'OGIG', 'CLCT', 'CNQ', 'CSX', 'EW', 'FLO', 'FXU', 'GENC', 'GM', 'GOL', 'HLX', 'INSP', 'IR', 'KKR', 'LFUS', 'LOMA', 'MAN',

In [43]:
#========================#
#      Ejercicio 3       #
#========================#


def sublist_to_string(listas):
    for i in range(len(listas)):
        listas[i] = ','.join(listas[i])
    return listas

listas = sublist_to_string(listasActivos())
listas[0]

'BWA,BTN,DLA,EPR,IBN,KBH,MED,MODN,NEWR,IVZ,OLN,PLAN,PNM,PSX,RH,SMAR,SPXC,TYL,UL,VEON,AAME,AAP,AAWW,AAXN,ABB,ABCB,ABG,ABMD,AAOI,AAXJ,AB,ABC,ABEV,ABM,ABR,ABTX,AC,ACAD,ACA,ACAM,ACC,ACCO,ACET,ACHC,ACIW,ACM,ACN,ACND,ACTG,ACWI,ACWX,ADMA,ADNT,ADPT,ADS,ADSW,ADVM,ACBI,ACCD,ACEL,ACGL,ACH,ACI,ACLS,ACMR,ACNB,ACND.U,ACU,ACWV,ADC,ADES,ADI,ADM,ADSK,ADTN,ADUS,AEG,AEHR,AEL,AEE,AEGN,AEIS,AEM,AEO,AER,AES,AEYE,AFL,AG,AGCO,AGD,AGEN,AGI,AGLE,AGNC,AGNCN,AGNCP,AGRO,AGS,AEP,AERI,AEY,AFB,AFG,AFIB,AFIN,AFMD,AFYA,AGCUU,AGFS,AGG,AGGY,AGIO,AGM,AGNCM,AGO,AGQ,AGX,AGTC,AGYS,AHC,AHH,AINV,AIQ,AIRG,AIRR,AIV,AIZP,AJRD,AKUS,ALCO,ALE,ALEX,ALG,ALGT,AI,AIG,AIMC,AIN,AIR,AIRI,AIRT,AIT,AJG,AJX,AKAM,AKBA,AKR,AL,ALC,ALDX,ALEC,ALGN,ALIM,ALKS,ALOT,ALK,ALLE,ALRM,ALRS,ALSN,ALTA,ALTR,ALXN,AMAG,AMAT,AMBC,AMCX,AMED,AMG,AMH,AMHC,AMJ,AMKR,AMN,AMNB,AMOV,AMRC,AMRK,ALRN,ALSK,ALTG,ALTM,ALUS,ALV,ALX,AM,AMAL,AMBA,AMCR,AME,AMEH,AMGN,AMK,AMLP,AMOT,AMP,AMPH,AMRB,AMRN'

In [36]:
#========================#
#      Ejercicio 4       #
#========================#


def getHistory(symbol, start, end, timeframe='1D'):    
    start = dt.datetime.strftime(start, format='%Y-%m-%dT%T-04:00')
    end = dt.datetime.strftime(end, format='%Y-%m-%dT%T-04:00')

    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params =  {'symbols' : symbol, 'start':start, 'end':end}
    endpoint = "https://data.alpaca.markets/v1/bars/"+timeframe
    r = requests.get(url = endpoint, headers =headers, params=params)
    js = r.json()
    
    tickers = symbol.split(",")
    dfs = {}
    for ticker in tickers:
        # Bloque por los errores cuando no hay datos
        try:
            dfs[ticker] = pd.DataFrame(js[ticker])
            dfs[ticker].t = pd.to_datetime(dfs[ticker].t, unit='s')
            dfs[ticker].set_index(dfs[ticker].t, inplace=True)
        except:
            pass
    return dfs

import datetime as dt
end =  dt.date.today() + dt.timedelta(days=1)
start = end - dt.timedelta(days=200)

data = getHistory(listas[0], start, end, timeframe='1D')

In [37]:
len(data)

200

In [38]:
data.keys()

dict_keys(['BWA', 'BTN', 'DLA', 'EPR', 'IBN', 'KBH', 'MED', 'MODN', 'NEWR', 'IVZ', 'OLN', 'PLAN', 'PNM', 'PSX', 'RH', 'SMAR', 'SPXC', 'TYL', 'UL', 'VEON', 'AAME', 'AAP', 'AAWW', 'AAXN', 'ABB', 'ABCB', 'ABG', 'ABMD', 'AAOI', 'AAXJ', 'AB', 'ABC', 'ABEV', 'ABM', 'ABR', 'ABTX', 'AC', 'ACAD', 'ACA', 'ACAM', 'ACC', 'ACCO', 'ACET', 'ACHC', 'ACIW', 'ACM', 'ACN', 'ACND', 'ACTG', 'ACWI', 'ACWX', 'ADMA', 'ADNT', 'ADPT', 'ADS', 'ADSW', 'ADVM', 'ACBI', 'ACCD', 'ACEL', 'ACGL', 'ACH', 'ACI', 'ACLS', 'ACMR', 'ACNB', 'ACND.U', 'ACU', 'ACWV', 'ADC', 'ADES', 'ADI', 'ADM', 'ADSK', 'ADTN', 'ADUS', 'AEG', 'AEHR', 'AEL', 'AEE', 'AEGN', 'AEIS', 'AEM', 'AEO', 'AER', 'AES', 'AEYE', 'AFL', 'AG', 'AGCO', 'AGD', 'AGEN', 'AGI', 'AGLE', 'AGNC', 'AGNCN', 'AGNCP', 'AGRO', 'AGS', 'AEP', 'AERI', 'AEY', 'AFB', 'AFG', 'AFIB', 'AFIN', 'AFMD', 'AFYA', 'AGCUU', 'AGFS', 'AGG', 'AGGY', 'AGIO', 'AGM', 'AGNCM', 'AGO', 'AGQ', 'AGX', 'AGTC', 'AGYS', 'AHC', 'AHH', 'AINV', 'AIQ', 'AIRG', 'AIRR', 'AIV', 'AIZP', 'AJRD', 'AKUS', 'ALCO'

In [39]:
#========================#
#      Ejercicio 5       #
#========================#


def signal(df, fast=9, slow=26, suavizado=9):
    df['ema_fast'] = df.c.ewm(span=fast).mean()
    df['ema_slow'] = df.c.ewm(span=slow).mean()
    df['macd'] = df.ema_fast - df.ema_slow
    df['signal'] = df.macd.ewm(span=suavizado).mean()
    df['histograma'] = df.macd - df.signal
    
    
    alert = ''
    if ( (float(df.iloc[-1:]['histograma']) > 0) & (float(df.iloc[-2:-1]['histograma']) < 0) ):
        alert = 'Subiendo'
    if ( (float(df.iloc[-1:]['histograma']) < 0) & (float(df.iloc[-2:-1]['histograma']) > 0) ):
        alert = 'Bajando'
        
    return alert

In [40]:
signal(data['nnnnn'], fast=9, slow=26, suavizado=9)

KeyError: 'nnnnn'

In [ ]:
#========================#
#      Ejercicio 6       #
#========================#

# Uso la funcion del ejercicio 3 y 2
# armo las listas de a 200 activos para traer la data lista por lista
listas = sublist_to_string(listasActivos())


import datetime as dt
end =  dt.date.today()  + dt.timedelta(days=1)
start = end - dt.timedelta(days=200)

subiendo = []
bajando = []

for i in range(len(listas)):
    
    # Para ver la evolución ya que tarda unos 2 minutos en correr
    print(i,'..',end='')
    
    # Uso la función del ejercicio 4
    data = getHistory(listas[i], start, end, timeframe='1D')
    for ticker in data:
        try:
            # Uso la función del ejercicio 5
            s = signal(data[ticker], fast=9, slow=26, suavizado=9)
            
            if s=='Subiendo':
                subiendo.append(ticker)
            if s=='Bajando':
                bajando.append(ticker)
        except:
            pass

In [ ]:
# Uso la función del ejercicio 1
longs_id, longs = createWatchlist('Longs', subiendo)
shorts_id, shorts = createWatchlist('Shorts', bajando)

print('Longs ID:',longs_id,'\nShorts ID:',shorts_id)
print('\n\nLongs Cantidad:',len(longs),'\nShorts Cantidad:',len(shorts))

In [ ]:
comprar = list(getWatchlist(longs_id).index.values)
vender = list(getWatchlist(shorts_id).index.values)
print(f'COMRAR:\n{comprar} \n\n VENDER:\n{vender}')

In [ ]:
end =  dt.date.today()  + dt.timedelta(days=1)
start = end - dt.timedelta(days=50)
preciosLong = getHistory(','.join(comprar), start, end, timeframe='15Min')

In [ ]:
for ticker in comprar:
    try:
        px = preciosLong[ticker].iloc[-1].c
        q = int(((200000)/len(comprar)) // px)
        orden = order(symbol=ticker, qty=q, side='buy', 
                      tipo='market',time_in_force='gtc',limit_price=None)
        
        print('Buy', round(px,2), ticker, end=', ')
    except:
        print(f'\n****Fallo en {ticker}****')

In [ ]:
end =  dt.date.today()  + dt.timedelta(days=1)
start = end - dt.timedelta(days=50)
preciosShort = getHistory(','.join(vender), start, end, timeframe='15Min')

In [ ]:
for ticker in vender:
    try:
        # lastQ=lastQuote(ticker)  px=(lastQ['askprice']+lastQ['bidprice'])/2 
        px = preciosShort[ticker].iloc[-1].c
        q = ((200000)/len(vender)) // px
        orden = order(symbol=ticker, qty=q, side='sell', 
                      tipo='market', time_in_force='gtc', limit_price=None)
        print('Sell', round(px,2), ticker, end=', ')
    except:
        print(f'****Fallo en {ticker}****')

# Bases de Datos introducción

* claves foraneas
* claves primarias

<img src='imagenes/BBDD.jpeg' width='600'>

Y así sucesivamente se pueden ir creando nuevas entidades que se van relacionando con otras

En este caso vamos a agregar a nuestro modelo de subyacentes y derivados, una entidad de "fondos" que podrian ser FCIs y "clientes" que podrian ser las cuentas de un Alyc

<img src='imagenes/BBDD2.jpeg' width='600'>

# XAMPP

## Configuracion Apache y puerto http

Configuraciones Iniciales, no necesariamente pero si al darrle "start" al Apache aparece un conflicto de puertos, hay que configurar un puerto diferente al 80 que es el default

<img src='imagenes/xampp_1.png' width=750>

<img src='imagenes/xampp_2.png' width=750>

<img src='imagenes/xampp_3.png' width=750>

## Configuracion MySQL y puerto de BBDD

<img src='imagenes/xampp_4.png' width=750>

<img src='imagenes/xampp_5.png' width=750>

## Configuracion general  XAMMP

<img src='imagenes/xampp_6.png' width=800>

<img src='imagenes/xampp_7.png' width='600'>

<img src='imagenes/xampp_8.png'  width='600'>

# Crear Base de datos

<img src='imagenes/xampp_9.png'  width='700'>

# Seteo Motor de BBDD

In [ ]:
# from sqlalchemy import create_engine

# Seteo el USER : PASS @ HOST / BBDD_NAME
# engine = create_engine('mysql+pymysql://root:@localhost/test')